In [1]:
import cv2
from picamera2 import Picamera2
from ultralytics import YOLO
import os
os.environ["DISPLAY"] = ":0"


In [2]:
# model_path = r"C:\Users\chena\Desktop\EZLift\model_training\runs\detect\pallets\weights\best.pt"
# model_path = '/home/pi/EZLift/model_training/runs/detect/pallets/weights/best_ncnn_model'
# model_path = '/home/pi/EZLift/model_training/runs/detect/pallets/weights/best_saved_model/best_float16.tflite'

# model_path = r"/home/pi/EZLift/model_training/runs/detect/pallets_256/weights/best.pt"
model_path = r"/home/pi/EZLift/model_training/runs/detect/pallets_256/weights/best_ncnn_model"

model = YOLO(model_path)

In [3]:
# model.export(format="ncnn")
# model.export(format="tflite")

In [ ]:
picam2 = Picamera2()
picam2.preview_configuration.main.size = (1920, 1080)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
# picam2.configure("preview")
# picam2.start()
picam2.start(show_preview=False)

while True:
    # Capture frame-by-frame
    frame = picam2.capture_array()

    frame = cv2.resize(frame, (320, 320), interpolation=cv2.INTER_NEAREST)
    # Run YOLO11 inference on the frame
    results = model(frame)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Display the resulting frame
    cv2.imshow("Camera", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()

[1:08:39.376478899] [3118]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[1:08:39.409840887] [3135]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[1:08:39.411647618] [3135]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[1:08:39.412342082] [3135]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media1 and ISP device /dev/media2
[1:08:39.412395191] [3135]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[1:08:39.425043086] [3118]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[1:08:39.426165557] [3135]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Loading /home/pi/EZLift/model_training/runs/detect/pallets/weights/best_ncnn_model for NCNN inference...

0: 640x640 (no detections), 492.8ms
Speed: 23.1ms preprocess, 492.8ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 460.5ms
Speed: 24.9ms preprocess, 460.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 458.8ms
Speed: 18.6ms preprocess, 458.8ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 464.2ms
Speed: 18.2ms preprocess, 464.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 460.3ms
Speed: 16.3ms preprocess, 460.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 462.8ms
Speed: 16.1ms preprocess, 462.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 487.7ms
Speed: 16.4ms preprocess, 487.7ms inferenc